***DOTA 2's 2017 The International*** I would like to revisit an eports event that I enjoyed the most using the OpenDota API

In [1]:
#Getting the league information
import requests
response = requests.get("https://api.opendota.com/api/leagues")
league_matches = response.json()

In [2]:
#Getting the league id
for league in league_matches : 
    if league['name'] == 'The International 2017' :
        league_id = league['leagueid']

In [3]:
#Getting league matches
match_ids = []
response = requests.get("https://api.opendota.com/api/leagues/" + str(league_id) + "/matches")
league_matches = response.json()
print("Number of Matches: ", len(league_matches))

Number of Matches:  540


In [4]:
#Converting hierarical data to dataframe
import pandas as pd
from pandas import json_normalize

league_df = json_normalize(league_matches)
league_df.head()

,match_id,match_seq_num,radiant_win,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,cluster,...,cosmetics.5693,cosmetics.5801,cosmetics.9068,cosmetics.6458,cosmetics.9459,cosmetics.9472,cosmetics.9475,cosmetics.4687,cosmetics.4987,cosmetics.9017
0,3372622939,2941273643,False,1502576268,1654,1568,1983,14,63,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3354058128,2925747249,False,1501791704,1946,1540,2036,3,63,113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3356189302,2927540359,False,1501890649,2658,512,1974,0,63,113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3353737069,2925514083,True,1501776112,3432,1796,1536,63,35,113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3358237109,2929243568,False,1501967991,1637,1590,2046,63,63,113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Filtering out unnecessary features
league_df.keys()
league_df = league_df[["match_id", "radiant_win", "duration", "start_time", "radiant_score", "dire_score"]]
league_df.columns = ["Match ID", "Radiant Win", "Duration (mins)", "Date", "Radiant Kills", "Dire Kills"]
league_df.head()

,Match ID,Radiant Win,Duration (mins),Date,Radiant Kills,Dire Kills
0,3372622939,False,1654,1502576268,13,29
1,3354058128,False,1946,1501791704,5,26
2,3356189302,False,2658,1501890649,13,44
3,3353737069,True,3432,1501776112,40,22
4,3358237109,False,1637,1501967991,11,28


In [6]:
#Converting Seconds to Minutes
league_df['Duration (mins)'] = league_df['Duration (mins)'] / 60
league_df.head()

,Match ID,Radiant Win,Duration (mins),Date,Radiant Kills,Dire Kills
0,3372622939,False,27.566667,1502576268,13,29
1,3354058128,False,32.433333,1501791704,5,26
2,3356189302,False,44.300000,1501890649,13,44
3,3353737069,True,57.200000,1501776112,40,22
4,3358237109,False,27.283333,1501967991,11,28


In [7]:
#Converting Unix timestamp
league_df['Date'] = pd.to_datetime(league_df['Date'],unit='s')
league_df.head()

,Match ID,Radiant Win,Duration (mins),Date,Radiant Kills,Dire Kills
0,3372622939,False,27.566667,2017-08-12 22:17:48,13,29
1,3354058128,False,32.433333,2017-08-03 20:21:44,5,26
2,3356189302,False,44.300000,2017-08-04 23:50:49,13,44
3,3353737069,True,57.200000,2017-08-03 16:01:52,40,22
4,3358237109,False,27.283333,2017-08-05 21:19:51,11,28


In [8]:
#Getting heros information
response = requests.get("https://api.opendota.com/api/heroes")
heros_json = response.json()
heros_df = json_normalize(heros_json)
heros_df.set_index("id", inplace = True)
heros_df.head()

,name,localized_name,primary_attr,attack_type,roles,legs
id,,,,,,
1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler, Carry]",2
3,npc_dota_hero_bane,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",4
4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Jungler, Nuker, Initiator]",2
5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker, Jungler]",2


In [9]:
#Getting Team & Hero Information
import time

Radiant_Team_Name = []
Radiant_Team_Memb = []
Radiant_Team_Hero = []

Dire_Team_Name = []
Dire_Team_Memb = []
Dire_Team_Hero = []

for match_id in league_df['Match ID'] : #Getting each Match breakdown
    RTM = []; DTM = []; RTH = []; DTH = []
    response = requests.get("https://api.opendota.com/api/matches/" + str(match_id))
        
    match = response.json()
    Radiant_Team_Name.append(match['radiant_team']['name'])
    Dire_Team_Name.append(match['dire_team']['name'])
    
    for player in match['players'] : 
        if player['player_slot'] <= 127 : 
            RTM.append(player['name'])
            RTH.append(heros_df['localized_name'].loc[player['hero_id']])
        else :
            DTM.append(player['name'])
            DTH.append(heros_df['localized_name'].loc[player['hero_id']])
    
    Radiant_Team_Memb.append(RTM)
    Radiant_Team_Hero.append(RTH)
    Dire_Team_Memb.append(DTM)
    Dire_Team_Hero.append(DTH)
    time.sleep(2.0) #Limited by Free API Constraints (60req/min)

In [10]:
#Adding Team Information
league_df['Radiant Name'] = Radiant_Team_Name
league_df['Radiant Players'] = Radiant_Team_Memb
league_df['Radiant Heros'] = Radiant_Team_Hero

league_df['Dire Name'] = Dire_Team_Name
league_df['Dire Players'] = Dire_Team_Memb
league_df['Dire Heros'] = Dire_Team_Hero

league_df.head()

,Match ID,Radiant Win,Duration (mins),Date,Radiant Kills,Dire Kills,Radiant Name,Radiant Players,Radiant Heros,Dire Name,Dire Players,Dire Heros
0,3372622939,False,27.566667,2017-08-12 22:17:48,13,29,Newbee,"[Faith, Sccc丶, Moogy, kpii, kaka]","[Crystal Maiden, Lina, Bloodseeker, Batrider, ...",Team Liquid,"[Miracle-, MinD_ContRoL, KuroKy, Gh, MATUMBAMAN]","[Troll Warlord, Nature's Prophet, Lich, Slarda..."
1,3354058128,False,32.433333,2017-08-03 20:21:44,5,26,HellRaisers,"[j4, MiLAN, Swiftending, 33, Emperor]","[Ancient Apparition, Clockwerk, Lifestealer, E...",OG,"[Fly, ana, JerAx, s4, N0tail]","[Phoenix, Ursa, Nyx Assassin, Enigma, Phantom ..."
2,3356189302,False,44.300000,2017-08-04 23:50:49,13,44,OG,"[Fly, ana, JerAx, s4, N0tail]","[Phoenix, Earthshaker, Treant Protector, Enigm...",INVICTUS GAMING,"[Q, OpGod, Xxs, Borax, BurNIng]","[Lich, Ember Spirit, Legion Commander, Earth S..."
3,3353737069,True,57.200000,2017-08-03 16:01:52,40,22,Newbee,"[Faith, Sccc丶, Moogy, kaka, kpii]","[Lich, Morphling, Silencer, Bounty Hunter, Leg...",OG,"[Fly, ana, JerAx, s4, N0tail]","[Winter Wyvern, Anti-Mage, Tusk, Clockwerk, Br..."
4,3358237109,False,27.283333,2017-08-05 21:19:51,11,28,LGD.Forever Young,"[ddc, - ah fu -, Super！, Monet, 剑来！]","[Visage, Spirit Breaker, Venomancer, Monkey Ki...",OG,"[Fly, ana, JerAx, s4, N0tail]","[Dazzle, Ember Spirit, Night Stalker, Magnus, ..."


In [11]:
#Rearranging Columns
cols = ["Match ID", "Date", "Duration (mins)", "Radiant Win", "Radiant Name", "Radiant Players", "Radiant Heros", "Radiant Kills", "Dire Name", "Dire Players", "Dire Heros", "Dire Kills"]
league_df = league_df[cols]
league_df.head()

,Match ID,Date,Duration (mins),Radiant Win,Radiant Name,Radiant Players,Radiant Heros,Radiant Kills,Dire Name,Dire Players,Dire Heros,Dire Kills
0,3372622939,2017-08-12 22:17:48,27.566667,False,Newbee,"[Faith, Sccc丶, Moogy, kpii, kaka]","[Crystal Maiden, Lina, Bloodseeker, Batrider, ...",13,Team Liquid,"[Miracle-, MinD_ContRoL, KuroKy, Gh, MATUMBAMAN]","[Troll Warlord, Nature's Prophet, Lich, Slarda...",29
1,3354058128,2017-08-03 20:21:44,32.433333,False,HellRaisers,"[j4, MiLAN, Swiftending, 33, Emperor]","[Ancient Apparition, Clockwerk, Lifestealer, E...",5,OG,"[Fly, ana, JerAx, s4, N0tail]","[Phoenix, Ursa, Nyx Assassin, Enigma, Phantom ...",26
2,3356189302,2017-08-04 23:50:49,44.300000,False,OG,"[Fly, ana, JerAx, s4, N0tail]","[Phoenix, Earthshaker, Treant Protector, Enigm...",13,INVICTUS GAMING,"[Q, OpGod, Xxs, Borax, BurNIng]","[Lich, Ember Spirit, Legion Commander, Earth S...",44
3,3353737069,2017-08-03 16:01:52,57.200000,True,Newbee,"[Faith, Sccc丶, Moogy, kaka, kpii]","[Lich, Morphling, Silencer, Bounty Hunter, Leg...",40,OG,"[Fly, ana, JerAx, s4, N0tail]","[Winter Wyvern, Anti-Mage, Tusk, Clockwerk, Br...",22
4,3358237109,2017-08-05 21:19:51,27.283333,False,LGD.Forever Young,"[ddc, - ah fu -, Super！, Monet, 剑来！]","[Visage, Spirit Breaker, Venomancer, Monkey Ki...",11,OG,"[Fly, ana, JerAx, s4, N0tail]","[Dazzle, Ember Spirit, Night Stalker, Magnus, ...",28


This is a basic dataframe with the Hero information and Player information along with Win scores. This can be further imporved by adding individual rank, role, hero damage, last hits, tower damage and so on....to improve and get a good statistic for the League!!! In case you are wondering, ***Team Liquid won the 2017 International!***